In [18]:
import xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

### Simple Case: Use simple xgb-regressor on entire training data

In [3]:
train_set = pd.read_csv(r'../data/train-preprocessed.csv')
test_set = pd.read_csv(r'../data/test-preprocessed.csv')

In [4]:
X_train = train_set.drop(['SalePrice'], axis = 1).values
y_train = train_set['SalePrice'].values

In [ ]:
reg = xgboost.XGBRegressor()
reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [6]:
y_pred = reg.predict(test_set)

In [7]:
y_pred

array([124207.586, 141063.67 , 185765.48 , ..., 154401.88 , 106872.75 ,
       207102.53 ], dtype=float32)

In [8]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv('../data/sample_submission_master.csv')
datasets = pd.concat([sub_df['Id'], pred], axis = 1)
datasets.columns = ['Id', 'SalePrice']
# datasets.to_csv('sample_submission.csv',index = False)

### Use validation set to evaluate performance before submission:

In [10]:
train_set = pd.read_csv(r'../data/train-preprocessed.csv')
test_set = pd.read_csv(r'../data/test-preprocessed.csv')

In [11]:
X_train = train_set.drop(['SalePrice'], axis = 1).values
y_train = train_set['SalePrice'].values

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [13]:
reg = xgboost.XGBRegressor()
reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
y_pred = reg.predict(X_val)
y_pred

array([153611.92 , 345751.47 , 109269.49 , 159112.56 , 329905.3  ,
        82253.51 , 215146.48 , 146728.53 ,  86224.89 , 132071.53 ,
       172813.16 , 123547.164, 118669.625, 211036.44 , 169960.33 ,
       126702.375, 205460.28 , 139001.06 , 126723.47 , 196679.8  ,
       151210.84 , 228225.45 , 171587.58 , 134412.   , 197245.73 ,
       160856.11 , 192447.27 , 103517.41 , 173304.28 , 213185.69 ,
       112225.82 , 240633.17 , 155268.27 , 115707.14 , 245915.28 ,
       150378.3  , 126189.14 , 204009.17 , 305101.5  , 125876.42 ,
       135943.73 , 241698.4  , 115234.836, 378550.53 , 132641.03 ,
       136344.92 , 111652.47 , 119040.84 , 246609.89 , 145918.56 ,
       114211.92 , 200929.1  , 115157.69 , 364562.9  , 139749.16 ,
       255543.62 , 207874.38 , 139344.97 , 149075.1  , 116787.66 ,
        64701.977, 161199.16 , 330078.3  , 325828.75 , 275660.66 ,
       226308.36 , 114198.76 , 309453.78 ,  90732.62 , 171359.72 ,
       121193.016, 120220.336, 116102.87 ,  76742.   , 493606.

In [19]:
def log_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

In [20]:
log_rmse(y_val, y_pred)

np.float64(0.14331872456114564)